<a href="https://colab.research.google.com/github/Kanagat01/db_practice/blob/master/parser_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install aiohttp

In [5]:
import asyncio
import json
import aiohttp
from aiohttp.client_exceptions import *
from bs4 import BeautifulSoup as BS


keys = ["IP адрес", "Версия", "Сайт сервера", "Группа ВК",
        "Дискорд", "Особенности", "Мини-игры", "Плагины", "Моды"]
results = []

In [6]:
async def fetch(session, url):
    async with session.get(url) as response:
        return await response.text()

In [7]:
async def scrape_data(title, link, banner):
    async with aiohttp.ClientSession() as session:
        try:
            info = {}
            info["Название сервера"] = title
            info["Баннер"] = banner
            html = await fetch(session, f"https://hotmc.ru{link}")
            soup = BS(html, "html.parser")
            desc = soup.select_one(
                "#server > div.section.server-description > p")
            desc = desc.text if desc is not None else ""
            info["Описание"] = desc.strip()
            try:
                images = soup.select(
                    ".section > #slider-screenshots > a.fancybox.img-block")
                photos = []
                for idx, href in enumerate([x.get("href") for x in images]):
                    try:
                        async with session.get(href) as response:
                            data = await response.read()
                            ext = "png" if "png" in href else "jpg"
                            filename = f"./screenshots/{title}_{idx}.{ext}"
                            with open(filename, "wb") as f:
                                f.write(data)
                            photos.append(filename)
                    except Exception as ex:
                        print(ex)
                info["Фотографии"] = ", ".join(photos)
            except:
                info["Фотографии"] = ""
            rows = soup.select("#server > form > div.form-group")
            for row in rows:
                key = row.select_one("label").text.strip().replace(":", "")
                if key in keys:
                    div = row.select_one("div")
                    if div.select_one("input") is not None:
                        info[key] = div.select_one("input").get("value")
                    elif div.select("div > a") is not None:
                        info[key] = ", ".join(
                            [x.text for x in div.select("div > a")])
                    elif div.select_one("a") is not None:
                        info[key] = div.select_one("a").text
            results.append(info)
            print(info["Название сервера"])
        except (ClientConnectorError, ServerDisconnectedError, ClientOSError, ClientResponseError, Exception) as ex:
            print(ex)
            with open("./exceptions.txt", "a", encoding="utf-8") as f:
                f.write(f"{title}_:_{link}_:_{banner}\n")

In [8]:
async def main():
    data = open("./links2.txt", encoding="utf-8").read().split("\n")

    i = 0
    while i * 100 <= len(data):
        tasks = []
        for x in data[i*100:(i+1)*100]:
            try:
                title, link, banner = x.split("_:_")
                tasks.append(scrape_data(title, link, banner))
            except Exception as ex:
                print(ex)
                with open("./exceptions.txt", "a", encoding="utf-8") as f:
                    f.write(f"{x}\n")
        await asyncio.gather(*tasks)
        with open("./info.json", "a", encoding="utf-8") as f:
            global results
            json.dump(results, f, indent=4, ensure_ascii=False)
            results = []
        i += 1

In [12]:
await main()

Выходные данные были обрезаны до нескольких последних строк (5000).
noodlefish.aternos.host:49592
65.108.21.152:25748
Ukraine State
95.158.46.192
horizonanarchy.net
linegrief.aternos.me
MedialCraft
superggfarteh.aternos.me
Toxic-Mine
MineMage | бесплатные кейсы
InMine
Sworld
45.93.200.11:25621
peamouth.aternos.host:40352
KorraWorld Стань аватаром
Political Earth
jooposol.aternos.me
LinuxCraft
lndanarchy.mclan.ru
colt.aternos.host:10970
palkacraft.ru
65.108.227.231:25828
TokyoParis | Магический
mythicali.aternos.me
Galurian
45.93.200.22:25655
CruxCraft
mc.anarhoplay.com
mc.nexcraft.me
luxyland
Gradient Role Play
deathmc.cf
CosmoWorld
95.217.53.176:25676
EndLand - Vanilla
6.tcp.eu.ngrok.io:14714
Сычевальск
158.69.123.184:25822
45.93.200.19:25578
51.83.212.202:25565
MarshMineRp
45.93.200.9:25598
135.181.126.138:25571
SofilaCraft
play.vanillaplus.net
4PDA Network
Открытие Апокалипсиса
Endercubes
UsualWorld
UltraMine #1
play.masedworld.net
45.93.200.11:25609
tapox.aternos.me
f5.gamely.pro:5